In [41]:
import csv
import math
import sys

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot 
import optuna

from sklearn import *

# Подготовка данных

In [42]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

target_labels = ["Оператор", "Есть ли Интернет", "Состав ДХ"]

y = train_data.loc[:, target_labels]
X = train_data.drop(target_labels, axis=1)

In [43]:
index_of_test_data_beginning = len(X.index)
X = pd.concat([X, test_data], ignore_index=True)

X.drop(["id", "hhid"], axis=1, inplace=True)

In [44]:
X.isna().sum()

Общее телесмотрение                            0
Телесмотрение ночью                            0
Телесмотрение утром                           25
Телесмотрение в рабочее время                  0
Телесмотрение в вечернее время                 0
Телесмотрение в прайм тайм                     0
Общее телесмотрение Пн-Пт                     25
Телесмотрение ночью Пн-Пт                      0
Телесмотрение утром Пн-Пт                      0
Телесмотрение в рабочее время Пн-Пт            0
Телесмотрение в вечернее время Пн-Пт           0
Телесмотрение в прайм тайм Пн-Пт               0
Общее телесмотрение в выходные                 0
Телесмотрение ночью в выходные                 0
Телесмотрение утром в выходные                 0
Телесмотрение в рабочее время в выходные       0
Телесмотрение вечером в выходные               0
Телесмотрение в прайм тайм в выходные          0
Просмотр жанра кино в рабочее время            0
Просмотр жанра кино в праздники                0
Просмотр жанра прикл

##### Начнем с заполнения пропусков для утреннего телесмотрения. Как вариант возможно проссуммировать телесмотрения в различное время суток и вычесть из общего телесмотрения.

In [45]:
col_names_for_tv_viewing = ["Телесмотрение утром", "Телесмотрение ночью", "Телесмотрение в рабочее время", "Телесмотрение в вечернее время", "Телесмотрение в прайм тайм", "Общее телесмотрение"]

indexes_of_rows_with_missed_values_for_morning_tv_viewing = X[col_names_for_tv_viewing[0]].isnull().where(lambda x: x).dropna().index

tv_viewing_per_part_of_day_and_in_total = X.loc[indexes_of_rows_with_missed_values_for_morning_tv_viewing, col_names_for_tv_viewing[1:]]

tv_viewing_in_total = tv_viewing_per_part_of_day_and_in_total[col_names_for_tv_viewing[-1]]
tv_viewing_per_part_of_day_and_in_total.drop(col_names_for_tv_viewing[-1], axis=1, inplace=True)

morning_tv_viewing_data = []
for index, row in tv_viewing_per_part_of_day_and_in_total.iterrows():
    morning_tv_viewing = tv_viewing_in_total[index] - sum(row)
    morning_tv_viewing_data.append(morning_tv_viewing)
morning_tv_viewings = pd.Series(data=morning_tv_viewing_data, index=tv_viewing_per_part_of_day_and_in_total.index)

X.loc[morning_tv_viewings.index, col_names_for_tv_viewing[0]] = morning_tv_viewings

X.loc[morning_tv_viewings.index, :]

,Общее телесмотрение,Телесмотрение ночью,Телесмотрение утром,Телесмотрение в рабочее время,Телесмотрение в вечернее время,Телесмотрение в прайм тайм,Общее телесмотрение Пн-Пт,Телесмотрение ночью Пн-Пт,Телесмотрение утром Пн-Пт,Телесмотрение в рабочее время Пн-Пт,...,Просмотр жанра спорт в праздники,Просмотр жанра музыка в рабочее время,Просмотр жанра музыка в праздники,Просмотр жанра образования в рабочее время,Просмотр жанра образование в праздники,Размер ДХ,Тип ресивера,ФО,Тип нас.пункта,Почтовый индекс
987,578,123,42.0,336,77,0,429.0,0,17,335,...,0,1,0,0,0,3.0,Ресивер с поддержкой HEVC,Приволжский федеральный округ,Город с численностью от 45 тыс. до 400 тыс.,617060
988,504,0,131.0,225,121,27,308.0,0,0,160,...,24,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Приволжский федеральный округ,Сельский населенный пункт,452020
989,409,110,0.0,282,17,0,408.0,109,0,282,...,0,0,0,3,0,2.0,Ресивер с поддержкой HDTV,Уральский федеральный округ,Город с численностью от 45 тыс. до 400 тыс.,622051
990,1517,0,146.0,1202,156,13,617.0,0,118,499,...,0,105,26,0,0,4.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,353417
991,49,0,0.0,49,0,0,0.0,0,0,0,...,0,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,353366
992,1047,77,160.0,507,123,180,76.0,76,0,0,...,0,0,0,0,0,3.0,Ресивер с поддержкой HEVC,Центральный федеральный округ,Город с численностью от 45 тыс. до 400 тыс.,143002
993,227,0,16.0,211,0,0,211.0,0,0,211,...,0,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Центральный федеральный округ,Сельский населенный пункт,396370
994,1690,3,123.0,905,425,234,704.0,2,53,300,...,10,0,0,0,0,1.0,Ресивер с поддержкой HEVC,Северо-Западный федеральный округ,Сельский населенный пункт,180560
995,411,0,134.0,248,29,0,159.0,0,129,30,...,0,0,0,0,0,4.0,Ресивер с поддержкой HEVC,Северо-Западный федеральный округ,Город с численностью менее 45 тыс.,169420
996,2164,431,265.0,1083,199,186,819.0,111,155,218,...,0,0,0,1,0,2.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,347660


##### Далее заполним пропуски в общем телесмотрении Пн-Пт. Оно вычисляется по аналогии, только на основе колонок, содержащих данные о телесмотрении Пн-Пт.

In [46]:
col_names_for_tv_viewing_in_work_days = ["Общее телесмотрение Пн-Пт", "Телесмотрение утром Пн-Пт", "Телесмотрение ночью Пн-Пт", "Телесмотрение в рабочее время Пн-Пт", "Телесмотрение в вечернее время Пн-Пт", "Телесмотрение в прайм тайм Пн-Пт"]

indexes_of_rows_with_missed_values_for_tv_viewing_in_total = X[col_names_for_tv_viewing_in_work_days[0]].isnull().where(lambda x: x).dropna().index

tv_viewing_per_part_of_day = X.loc[indexes_of_rows_with_missed_values_for_tv_viewing_in_total, col_names_for_tv_viewing_in_work_days[1:]]

tv_viewing_in_total_data = []
for index, row in tv_viewing_per_part_of_day.iterrows():
    tv_viewing_in_total = sum(row)
    tv_viewing_in_total_data.append(tv_viewing_in_total)
tv_viewings_in_total = pd.Series(data=tv_viewing_in_total_data, index=tv_viewing_per_part_of_day.index)

X.loc[tv_viewings_in_total.index, col_names_for_tv_viewing_in_work_days[0]] = tv_viewings_in_total

X.loc[tv_viewings_in_total.index, :]

,Общее телесмотрение,Телесмотрение ночью,Телесмотрение утром,Телесмотрение в рабочее время,Телесмотрение в вечернее время,Телесмотрение в прайм тайм,Общее телесмотрение Пн-Пт,Телесмотрение ночью Пн-Пт,Телесмотрение утром Пн-Пт,Телесмотрение в рабочее время Пн-Пт,...,Просмотр жанра спорт в праздники,Просмотр жанра музыка в рабочее время,Просмотр жанра музыка в праздники,Просмотр жанра образования в рабочее время,Просмотр жанра образование в праздники,Размер ДХ,Тип ресивера,ФО,Тип нас.пункта,Почтовый индекс
1000,1935,151,207.0,1117,221,239,304.0,4,0,79,...,0,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,353423
1001,264,106,0.0,158,0,0,40.0,16,0,24,...,0,0,0,0,0,3.0,Ресивер с поддержкой HDTV,Уральский федеральный округ,Сельский населенный пункт,456402
1002,1265,18,441.0,712,94,0,0.0,0,0,0,...,0,0,0,0,0,6.0,Ресивер с поддержкой HDTV,Приволжский федеральный округ,Город с численностью менее 45 тыс.,606912
1003,3156,113,550.0,1888,267,338,1473.0,60,185,881,...,0,0,0,0,22,3.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,347512
1004,1575,0,0.0,709,484,382,806.0,0,0,372,...,0,0,0,0,0,3.0,Ресивер с поддержкой HEVC,Приволжский федеральный округ,Сельский населенный пункт,462800
1005,1855,440,480.0,935,0,0,272.0,76,51,145,...,0,0,0,0,0,4.0,Ресивер с поддержкой HDTV,Приволжский федеральный округ,Город с численностью менее 45 тыс.,423520
1006,1874,0,233.0,1470,149,22,1080.0,0,233,847,...,0,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Центральный федеральный округ,Город с численностью менее 45 тыс.,171900
1007,1157,0,127.0,915,115,0,506.0,0,127,379,...,0,0,0,0,0,5.0,Ресивер с поддержкой HEVC,Южный федеральный округ,Сельский населенный пункт,353993
1008,701,9,183.0,508,1,0,623.0,9,183,431,...,0,0,0,0,0,6.0,Ресивер с поддержкой HEVC,Северо-Кавказский федеральный округ,Сельский населенный пункт,366501
1009,686,175,194.0,317,0,0,660.0,175,186,299,...,0,0,0,0,0,2.0,Ресивер с поддержкой HEVC,Центральный федеральный округ,Сельский населенный пункт,152861


##### Для повышения качества предсказания состава ДХ не лишено смысла свести данную задачу к предсказанию наличия членов семьи в возрасте от 55 и до 18 лет по отдельности.

In [47]:
family = y["Состав ДХ"].apply(lambda x: x.split(" и "))
y.drop("Состав ДХ", axis=1, inplace=True)

elder_generation  = family.transform(lambda x: x[0])
children = family.transform(lambda x: x[1])

y["Члены семьи в возрасте от 55 лет"] = elder_generation
y["Члены семьи до 18 лет"] = children

In [48]:
# Держим несколько кодировщиков для дальнейшего вызова inverse_transform().
encoder_per_label = {
    "Оператор": preprocessing.LabelEncoder(),
    "Есть ли Интернет": preprocessing.LabelEncoder(),
    "Члены семьи в возрасте от 55 лет": preprocessing.LabelEncoder(),
    "Члены семьи до 18 лет": preprocessing.LabelEncoder()
}

for label in encoder_per_label.keys():
    y[label] = encoder_per_label[label].fit_transform(y[label])

predictor_col_names = list(encoder_per_label.keys())

##### В частности закодируем тип населенного пункта для заполнения пропусков.

In [49]:
categorical_encoder = preprocessing.LabelEncoder()

X.loc[:, ["Тип ресивера", "Тип нас.пункта"]] = X.loc[:, ["Тип ресивера", "Тип нас.пункта"]].apply(categorical_encoder.fit_transform)
X["Тип нас.пункта"].where(lambda x: x != 4, other=np.nan, inplace=True)
X["Тип ресивера"] = X["Тип ресивера"].astype(int)

# X.loc[:, ["Тип ресивера", "ФО", "Тип нас.пункта"]]
X.loc[:, ["Тип ресивера", "Тип нас.пункта"]]

,Тип ресивера,Тип нас.пункта
0,0,2
1,1,1
2,0,0
3,1,3
4,0,3
...,...,...
4050,0,0
4051,1,3
4052,0,0
4053,1,0


In [50]:
standard_scaler = preprocessing.StandardScaler()
tv_viewing_data = X.drop(["Размер ДХ", "Тип ресивера", "Тип нас.пункта", "ФО", "Почтовый индекс"], axis=1)

X.loc[:, tv_viewing_data.columns] = standard_scaler.fit_transform(tv_viewing_data)

In [51]:
scaler = preprocessing.MinMaxScaler()
post_index = X["Почтовый индекс"].to_numpy().reshape(-1, 1)

X["Почтовый индекс"] = scaler.fit_transform(post_index)

X["Почтовый индекс"]

0       0.029493
1       0.073988
2       0.056808
3       0.073300
4       0.027209
          ...   
4050    0.024771
4051    0.050144
4052    0.056676
4053    0.106202
4054    0.027834
Name: Почтовый индекс, Length: 4055, dtype: float64

In [52]:
federal_district_one_hot_encoded = pd.get_dummies(X.loc[:, ["ФО"]]).astype(int)
X = pd.concat([X, federal_district_one_hot_encoded], axis=1)
X.drop(["ФО"], axis=1, inplace=True)

##### Далее заполнение пропусков уже не такое очевидное. Поэтому будем ориентироваться на ближайших соседей с отобранными вручную признаками.

In [53]:
# n_neighbors = sqrt(number_of_samples)/2
imputer = impute.KNNImputer(n_neighbors=28)

In [54]:
cols_for_household_size_imputation = [*tv_viewing_data.columns, "Размер ДХ"]
data_for_household_size_imputation = X.loc[:, cols_for_household_size_imputation]

indexes_of_rows_with_missing_values_for_household_size= X.loc[:, "Размер ДХ"].isna().where(lambda x: x).dropna().index
rows_with_missing_values_for_household_size = data_for_household_size_imputation.loc[indexes_of_rows_with_missing_values_for_household_size, :]

rows_without_missing_values_for_household_size = data_for_household_size_imputation.loc[X["Размер ДХ"].notna().where(lambda x: x).dropna().index, :]

imputer.fit(rows_without_missing_values_for_household_size)
# 
imputed_data = imputer.transform(rows_with_missing_values_for_household_size)

index_of_household_column = data_for_household_size_imputation.columns.get_loc("Размер ДХ")
for imputed_row  in imputed_data:
    imputed_row[index_of_household_column] = round(imputed_row[index_of_household_column])

X.loc[indexes_of_rows_with_missing_values_for_household_size, cols_for_household_size_imputation] = imputed_data

X.loc[indexes_of_rows_with_missing_values_for_household_size]

,Общее телесмотрение,Телесмотрение ночью,Телесмотрение утром,Телесмотрение в рабочее время,Телесмотрение в вечернее время,Телесмотрение в прайм тайм,Общее телесмотрение Пн-Пт,Телесмотрение ночью Пн-Пт,Телесмотрение утром Пн-Пт,Телесмотрение в рабочее время Пн-Пт,...,Тип ресивера,Тип нас.пункта,Почтовый индекс,ФО_Приволжский федеральный округ,ФО_Северо-Западный федеральный округ,ФО_Северо-Кавказский федеральный округ,ФО_Сибирский федеральный округ,ФО_Уральский федеральный округ,ФО_Центральный федеральный округ,ФО_Южный федеральный округ
2004,1.032353,-0.527804,1.444114,0.968233,1.974052,0.069158,0.794548,-0.559718,1.688401,0.471173,...,0,2,0.022895,0,0,0,0,0,1,0
2005,2.101516,2.947383,1.933736,1.676248,1.304999,-0.419855,1.558508,4.523572,1.369706,0.477206,...,0,3,0.064648,0,0,0,0,0,1,0
2006,-0.879289,-0.596790,-0.098657,-1.164850,-0.121472,-0.206695,-0.696197,-0.475465,0.832477,-0.988701,...,1,1,0.106638,0,0,0,1,0,0,0
2007,-0.088573,-0.644218,-0.380420,0.291853,0.149936,-0.532704,0.002706,-0.559718,-0.797422,0.447043,...,1,3,0.073178,1,0,0,0,0,0,0
2008,-0.695443,-0.644218,-0.934709,-0.791259,0.187807,0.426514,-0.746384,-0.559718,-0.815633,-0.696123,...,1,3,0.057855,0,0,1,0,0,0,0
2009,0.365688,-0.066458,0.224678,0.742271,-0.197214,-0.532704,0.538036,-0.391211,0.022080,1.219208,...,1,0,0.063143,0,0,0,0,0,1,0
2010,-1.082770,-0.644218,-0.934709,-1.155812,-0.197214,-0.394777,-0.846758,-0.559718,-0.815633,-1.030929,...,0,3,0.066459,1,0,0,0,0,0,0
2011,0.705714,2.192845,1.753592,0.114096,-0.556987,-0.532704,1.489733,3.746568,1.715718,0.772800,...,1,3,0.073310,1,0,0,0,0,0,0
2012,1.159974,0.994207,-0.551326,1.254452,1.614278,0.627134,0.285241,1.228327,-0.496938,-0.056674,...,1,3,0.057855,0,0,1,0,0,0,0
2013,0.047973,-0.626972,-0.860804,0.569033,-0.077289,0.125583,-0.365333,-0.522272,-0.806527,0.106205,...,1,3,0.073191,1,0,0,0,0,0,0


In [55]:
cols_for_type_of_inhabited_locality_imputation = [*federal_district_one_hot_encoded, "Почтовый индекс", "Тип нас.пункта"]
data_for_type_of_inhabited_locality_imputation = X.loc[:, cols_for_type_of_inhabited_locality_imputation]

indexes_of_rows_with_missing_values_for_type_of_inhabited_locality = data_for_type_of_inhabited_locality_imputation.loc[:, "Тип нас.пункта"].isna().where(lambda x: x).dropna().index

rows_with_missing_values_for_type_of_inhabited_locality = data_for_type_of_inhabited_locality_imputation.loc[indexes_of_rows_with_missing_values_for_type_of_inhabited_locality, :]
#
rows_without_missing_values_for_type_of_inhabited_locality = data_for_type_of_inhabited_locality_imputation.loc[X["Тип нас.пункта"].notna().where(lambda x: x).dropna().index, :]

imputer.fit(rows_without_missing_values_for_type_of_inhabited_locality)
# 
imputed_data = imputer.transform(rows_with_missing_values_for_type_of_inhabited_locality)

index_of_type_of_inhabited_locality_column = data_for_type_of_inhabited_locality_imputation.columns.get_loc("Тип нас.пункта")
for imputed_row  in imputed_data:
    imputed_row[index_of_type_of_inhabited_locality_column] = round(imputed_row[index_of_type_of_inhabited_locality_column])

X.loc[indexes_of_rows_with_missing_values_for_type_of_inhabited_locality, cols_for_type_of_inhabited_locality_imputation] = imputed_data

X.loc[indexes_of_rows_with_missing_values_for_type_of_inhabited_locality]

,Общее телесмотрение,Телесмотрение ночью,Телесмотрение утром,Телесмотрение в рабочее время,Телесмотрение в вечернее время,Телесмотрение в прайм тайм,Общее телесмотрение Пн-Пт,Телесмотрение ночью Пн-Пт,Телесмотрение утром Пн-Пт,Телесмотрение в рабочее время Пн-Пт,...,Тип ресивера,Тип нас.пункта,Почтовый индекс,ФО_Приволжский федеральный округ,ФО_Северо-Западный федеральный округ,ФО_Северо-Кавказский федеральный округ,ФО_Сибирский федеральный округ,ФО_Уральский федеральный округ,ФО_Центральный федеральный округ,ФО_Южный федеральный округ
200,1.367916,0.442316,1.878307,0.936599,1.437547,1.091069,0.225760,0.704083,2.444164,-0.406561,...,1,3.0,0.059581,0,0,1,0,0,0,0
201,-0.761485,-0.644218,-0.814613,-0.467380,-0.834707,-0.532704,-1.075389,-0.559718,-0.815633,-1.030929,...,1,3.0,0.074798,1,0,0,0,0,0,0
202,-0.002005,-0.359650,-0.214134,0.519322,-0.834707,-0.532704,0.723914,0.039418,0.604837,1.225240,...,1,1.0,0.024684,0,0,0,0,0,1,0
203,-0.323289,1.429683,-0.089419,-0.688823,-0.834707,-0.532704,-0.547494,0.573023,-0.815633,-0.539277,...,0,2.0,0.107228,0,0,0,1,0,0,0
204,-0.782012,-0.644218,-0.925471,-0.465874,-0.834707,-0.532704,-0.579094,-0.559718,-0.797422,-0.231617,...,1,2.0,0.023151,0,0,0,0,0,1,0
205,1.836456,1.214101,0.451013,1.411119,2.087664,2.576916,1.906101,0.544938,1.633767,2.142186,...,0,1.0,0.024771,0,0,0,0,0,1,0
206,-1.187187,-0.644218,-0.934709,-1.146773,-0.834707,-0.532704,-1.064236,-0.559718,-0.815633,-1.012831,...,0,3.0,0.101404,0,0,0,0,1,0,0
207,0.366580,-0.501934,0.820539,0.853746,-0.834707,-0.532704,1.010167,-0.381850,0.841582,1.747055,...,1,3.0,0.057207,0,0,0,0,0,0,1
208,-0.468759,-0.644218,-0.371182,-0.117892,-0.834707,-0.532704,0.513872,-0.559718,0.286142,1.183013,...,1,3.0,0.063547,0,0,0,0,0,1,0
209,0.021199,-0.644218,-0.348087,0.456052,0.194119,-0.532704,0.307547,-0.559718,0.340775,0.609921,...,1,1.0,0.024745,0,0,0,0,0,1,0


In [56]:
X.drop([*federal_district_one_hot_encoded.columns, "Почтовый индекс"], axis=1, inplace=True)

In [57]:
type_of_inhabited_locality_one_hot_encoded = pd.get_dummies(X.loc[:, ["Тип нас.пункта"]]).astype(int)

X = pd.concat([X, type_of_inhabited_locality_one_hot_encoded], axis=1)
X.drop(["Тип нас.пункта"], axis=1, inplace=True)

In [58]:
# np.set_printoptions(threshold=sys.maxsize)
# 
# train_data = pd.concat((X.iloc[:index_of_test_data_beginning], y), axis=1)
# 
# outlier_detector = neighbors.LocalOutlierFactor(n_neighbors=28)
# 
# assumptions_about_outliers = outlier_detector.fit_predict(train_data)
# 
# indexes_of_outliers = np.where(assumptions_about_outliers == -1)
# train_data.drop(index=indexes_of_outliers[0], inplace=True)
# 
# y = train_data.loc[:, y.columns]
# X_final_test = X.iloc[index_of_test_data_beginning:]
# X = train_data.drop(y.columns, axis=1)

In [59]:
X_final_test = X.iloc[index_of_test_data_beginning:]
X = X.iloc[:index_of_test_data_beginning]

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X,
    y,
    test_size=.15,
    random_state=42
)

# Обучение

##### В качестве модели машинного обучения были испробованы: логистическая регрессия, наивный Байес, многослойный персептрон, машина опорных векторов, случайный лес и чрезвычайно случайный лес. На основе заданных вручную пространств гиперпараметров для каждого алгоритма была найдена наилучшая конфигурация. По итогам лучшим оказался чрезвычайно случайный лес, хоть и с незначительным отрывом. 

In [60]:
def optimization_objective(trial):
    hyperparameter_max_depth_range = trial.suggest_int(name="max_depth", low=2, high=128, log=True)
    hyperparameter_n_estimators_range = trial.suggest_int(name="n_estimators", low=30, high=500, log=True)
    hyperparameter_min_samples_leaf_range = trial.suggest_int(name="min_samples_leaf", low=1, high=3)
    hyperparameter_criterion_range = trial.suggest_categorical(name="criterion", choices=["gini", "entropy"])
    hyperparameter_min_samples_split_range = trial.suggest_int(name="min_samples_split", low=2, high=6, log=True)
    hyperparameter_max_features_range = trial.suggest_categorical(name="max_features", choices=["sqrt", "log2", None])

    classifier_tuned_with_random_hyperparameter_values = ensemble.ExtraTreesClassifier(
        n_estimators=hyperparameter_n_estimators_range,
        criterion=hyperparameter_criterion_range,
        max_depth=hyperparameter_max_depth_range,
        min_samples_split=hyperparameter_min_samples_split_range,
        min_samples_leaf=hyperparameter_min_samples_leaf_range,
        max_features=hyperparameter_max_features_range,
        warm_start=True,
        random_state=42,

    )
    multilabel_classifier_ =multioutput.ClassifierChain(classifier_tuned_with_random_hyperparameter_values)

    multilabel_classifier_.fit(X_train, y_train)
    # classifier_chain.score
    predictions_col_wise_ = multilabel_classifier_.predict(X_test).T
    
    # mean_metric_scores = {"f1": 0, "cohen_kappa": 0, "accuracy": 0}
    # for i_, (predictions_, label_) in enumerate(zip(predictions_col_wise_, y_test.columns)):
    #     actual_values_ = y_test[label_]
    # 
    #     accuracy_score_ = metrics.accuracy_score(actual_values_, predictions_)
    #     mean_metric_scores["accuracy"] += accuracy_score_
    # 
    #     f1_score_ = metrics.f1_score(actual_values_, predictions_)
    #     mean_metric_scores["f1"] += f1_score_
    # 
    #     cohen_kappa_score_ = metrics.cohen_kappa_score(actual_values_, predictions_)
    #     mean_metric_scores["cohen_kappa"] += cohen_kappa_score_
    cohen_kappa_scores = []
    for i_, (predictions_, label_) in enumerate(zip(predictions_col_wise_, y_test.columns)):
        actual_values_ = y_test[label_]
        cohen_kappa_score_ = metrics.cohen_kappa_score(actual_values_, predictions_)
        cohen_kappa_scores.append(cohen_kappa_score_)
    # for metric_name in mean_metric_scores.keys():
    #     mean_metric_scores.update({metric_name: mean_metric_scores.get(metric_name) / 4})
    return tuple(cohen_kappa_scores)

##### Найденная конфигурация может не совпасть с представленной ниже, поскольку точные параметры оптимизационного кода были утрачены в ходе ряда экспериментов. Закомментировано для удобства запуска.

In [61]:
# study = optuna.create_study(directions=["maximize", "maximize", "maximize", "maximize"])
# 
# study.optimize(optimization_objective, n_trials=200)
# 
# trial_with_highest_mean_metric_scores = max(study.best_trials, key=lambda x: x.values[1])
# print(trial_with_highest_mean_metric_scores.params)
# print(trial_with_highest_mean_metric_scores.values)

##### Класс ClassifierChain позволяет учесть зависимость между целевыми признаками в процессе предсказания, поэтому имеет смысл внедрить его в наш пайплайн. Также для повышения метрических показателей предсказания для членов семьи до 18 лет были произведены независимо от других целевых признаков.

In [62]:
best_found_model = ensemble.ExtraTreesClassifier(
    max_depth=64,
    n_estimators=59,
    min_samples_leaf=1,
    criterion='gini',
    min_samples_split=2,
    max_features='sqrt',
    warm_start=True,
    random_state=42
)

multilabel_classifier =multioutput.ClassifierChain(
    best_found_model,
    random_state=42
)
# 
multilabel_classifier.fit(X_train, y_train.loc[:, ["Оператор", "Есть ли Интернет", "Члены семьи в возрасте от 55 лет"]])
multilabel_predictions = multilabel_classifier.predict(X_test)

best_found_model.fit(X_train, y_train.loc[:, "Члены семьи до 18 лет"])

predictions_col_wise = np.concatenate((multilabel_predictions, best_found_model.predict(X_test).reshape(-1, 1)), axis=1).T

# Метрики

In [63]:
scores_per_metric = pd.DataFrame(columns=predictor_col_names, index=["accuracy","f1", "cohen_kappa"])
for i, (assumptions_about_outliers, predictor_col_name) in enumerate(zip(predictions_col_wise, predictor_col_names)):
    actual_values = y_test[predictor_col_name]

    accuracy_score = metrics.accuracy_score(actual_values, assumptions_about_outliers)
    f1_score = metrics.f1_score(actual_values, assumptions_about_outliers)
    cohen_kappa_score = metrics.cohen_kappa_score(actual_values, assumptions_about_outliers)

    scores_per_metric.loc["accuracy", predictor_col_name] = accuracy_score
    scores_per_metric.loc["f1", predictor_col_name, ] = f1_score
    scores_per_metric.loc["cohen_kappa", predictor_col_name] = cohen_kappa_score

scores_per_metric

,Оператор,Есть ли Интернет,Члены семьи в возрасте от 55 лет,Члены семьи до 18 лет
accuracy,0.958932,0.874743,0.895277,0.960986
f1,0.977477,0.826211,0.904673,0.964083
cohen_kappa,0.746248,0.728844,0.788573,0.92139


##### И напоследок получаем предсказания для тестовой части выборки. 

In [64]:
final_predictions_col_wise = np.concatenate((multilabel_classifier.predict(X_final_test), best_found_model.predict(X_final_test).reshape(-1, 1)), axis=1).T

final_predictions_per_label = {}
for i, label in enumerate(y_test.columns):
    final_predictions_per_label[label] = final_predictions_col_wise[i].astype(int)

##### Также приведем целевые признаки в исходный вид.

In [65]:
predictions_per_label_in_categorical_form =  np.empty(shape=(len(final_predictions_per_label.keys()), len(list(final_predictions_per_label.values())[0])), dtype=object)
for i, label in enumerate(encoder_per_label.keys()):
    predictions_per_label_in_categorical_form[i] = encoder_per_label[label].inverse_transform(final_predictions_per_label[label])

predictions_for_household_structure = list(map(
    " и ".join,
    zip(
        predictions_per_label_in_categorical_form[-2],
        predictions_per_label_in_categorical_form[-1]
    )
))

predictions_per_label_in_final_format = np.empty(shape=(len(target_labels), len(predictions_per_label_in_categorical_form[0])), dtype=object)
for i in range(len(target_labels)-1):
    predictions_per_label_in_final_format[i] = predictions_per_label_in_categorical_form[i]
predictions_per_label_in_final_format[2] = predictions_for_household_structure

In [66]:
predictions_per_row = predictions_per_label_in_final_format.T

predictions_per_row_in_final_format = pd.DataFrame(predictions_per_row, columns=target_labels)
predictions_per_row_in_final_format.insert(loc=0, column="id", value=range(1, len(predictions_per_row_in_final_format.index) + 1))

with open("submission/predictions.csv", "w+") as submission_file:
    writer = csv.writer(submission_file)
    writer.writerow(predictions_per_row_in_final_format.columns)
    writer.writerows(predictions_per_row_in_final_format.to_numpy())

# Обсуждение

##### Что хотелось бы сделать будь больше времени:
##### 1. Детальный анализ состава данных.
##### 2. Дальнейшая предобработка данных: поиск аномалий и автоматический отбор входных признаков.
##### 3. Подбор большего числа моделей машинного обучения.
 